In [8]:
from langchain_community.document_loaders import AsyncChromiumLoader
from langchain_community.document_transformers import BeautifulSoupTransformer
from langchain_openai import ChatOpenAI
#from langchain.chains import create_extraction
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain.chains import create_extraction_chain
import pprint
import asyncio
import nest_asyncio

from dotenv import load_dotenv
load_dotenv(".env")

True

In [13]:
import asyncio
from langchain_community.document_loaders import AsyncChromiumLoader
from langchain_community.document_transformers import BeautifulSoupTransformer
from playwright.async_api import async_playwright

async def load_and_transform():
    async with async_playwright() as p:
        # Ensure playwright is correctly set up
        browser = await p.chromium.launch()
        page = await browser.new_page()
        await page.goto("https://www.wsj.com")
        content = await page.content()
        await browser.close()
        
        # Apply BeautifulSoup transformation if needed
        transformer = BeautifulSoupTransformer()
        soup = transformer.transform(content)

        print(soup)

# Run the async function
asyncio.run(load_and_transform())


Task exception was never retrieved
future: <Task finished name='Task-18' coro=<Connection.run() done, defined at c:\Users\chris\Coding\ragvsgrag\venv\Lib\site-packages\playwright\_impl\_connection.py:265> exception=NotImplementedError()>
Traceback (most recent call last):
  File "C:\Program Files\WindowsApps\PythonSoftwareFoundation.Python.3.11_3.11.2544.0_x64__qbz5n2kfra8p0\Lib\asyncio\tasks.py", line 277, in __step
    result = coro.send(None)
             ^^^^^^^^^^^^^^^
  File "c:\Users\chris\Coding\ragvsgrag\venv\Lib\site-packages\playwright\_impl\_connection.py", line 272, in run
    await self._transport.connect()
  File "c:\Users\chris\Coding\ragvsgrag\venv\Lib\site-packages\playwright\_impl\_transport.py", line 135, in connect
    raise exc
  File "c:\Users\chris\Coding\ragvsgrag\venv\Lib\site-packages\playwright\_impl\_transport.py", line 123, in connect
    self._proc = await asyncio.create_subprocess_exec(
                 ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "C:\Pr

NotImplementedError: 

In [ ]:
#### SCRAPING WITH EXTRACTION

llm = ChatOpenAI(temperature=0, model="gpt-3.5-turbo-0613")

schema = {
    "properties": {
        "header": {"type": "string"},
        "content": {"type": "string"},
    },
    "required": ["header", "content"],
}
def extract(content: str, schema: dict):
    return create_extraction_chain(schema=schema, llm=llm).run(content)

def scrape_with_playwright(urls, schema):
    loader = AsyncChromiumLoader(urls)
    docs = loader.load()
    bs_transformer = BeautifulSoupTransformer()
    docs_transformed = bs_transformer.transform_documents(
        docs, tags_to_extract=["span"]
    )
    print("Extracting content with LLM")

    # Grab the first 1000 tokens of the site
    splitter = RecursiveCharacterTextSplitter.from_tiktoken_encoder(
        chunk_size=1000, chunk_overlap=0
    )
    splits = splitter.split_documents(docs_transformed)

    # Process the first split
    extracted_content = extract(schema=schema, content=splits[0].page_content)
    pprint.pprint(extracted_content)
    return extracted_content

urls = ["https://www.natwest.com/life-moments/managing-your-money/top-money-saving-tips.html"]
print(extracted_content = scrape_with_playwright(urls, schema=schema))

In [ ]:
import os
import pprint
from playwright.sync_api import sync_playwright
from bs4 import BeautifulSoup
from dotenv import load_dotenv

# Load environment variables
load_dotenv(".env")

def load_page_content(url):
    with sync_playwright() as p:
        browser = p.chromium.launch()
        page = browser.new_page()
        page.goto(url)
        content = page.content()
        browser.close()
        return content

def transform_content(html_content):
    soup = BeautifulSoup(html_content, 'html.parser')
    extracted_elements = soup.find_all(['h2', 'h3', 'p', 'li'])
    return '\n'.join([element.get_text() for element in extracted_elements])

def main():
    url = "https://www.natwest.com/life-moments/managing-your-money/top-money-saving-tips.html"
    html_content = load_page_content(url)
    transformed_content = transform_content(html_content)
    pprint.pprint(transformed_content[:30000])

if __name__ == "__main__":
    main()
